# Этот скрипт обнуляет все показы на частоте 6+ и переносит их на частоты 2 и 3 в соотношении 75 на 25

In [23]:
import pandas as pd
import numpy as np

# 1. Открытие и чтение CSV файла
df = pd.read_csv('old_frequency_by_placements_1000248234_2025-04-01_2025-06-01_ver_7d02da9e24f7.csv')
# Приведение Frequency к числовому типу
df['Frequency'] = df['Frequency'].replace('100+', 100)
df['Frequency'] = df['Frequency'].astype(int) 

# 2. Вывод списка Site Name и выбор
unique_sites = df['Site Name'].unique()
print("Доступные Site Name:")
for i, site in enumerate(unique_sites):
    print(f"{i + 1}. {site}")

selected_indices = input("Введите номера Site Name, с которыми будем работать (через запятую): ")
selected_indices = [int(i.strip()) - 1 for i in selected_indices.split(',')]
selected_sites = [unique_sites[i] for i in selected_indices]

# 3. Перераспределение показов
df_copy = df.copy()

for site in selected_sites:
    site_df = df_copy[df_copy['Site Name'] == site]
    
    for placement in site_df['Placement Name'].unique():
        placement_mask = (df_copy['Site Name'] == site) & (df_copy['Placement Name'] == placement)

        freq6plus_mask = placement_mask & (df_copy['Frequency'] >= 6)
        impressions_to_move = df_copy.loc[freq6plus_mask, 'Impressions'].sum()

        if impressions_to_move > 0:
            # Обнуляем показы на частоте 6+
            df_copy.loc[freq6plus_mask, 'Impressions'] = 0

            # Распределяем показы
            freq2_mask = placement_mask & (df_copy['Frequency'] == 2)
            freq3_mask = placement_mask & (df_copy['Frequency'] == 3)

            # Сколько надо добавить на частоту 2 и 3
            freq2_add = round(impressions_to_move * 0.75)
            freq3_add = impressions_to_move - freq2_add  # Остаток идет на частоту 3

            # Добавляем к существующим показам
            if df_copy.loc[freq2_mask].shape[0] > 0:
                df_copy.loc[freq2_mask, 'Impressions'] += freq2_add
            else:
                # Если строки с частотой 2 нет, создаем новую
                new_row = df_copy[placement_mask].iloc[0].copy()
                new_row['Frequency'] = 2
                new_row['Impressions'] = freq2_add
                df_copy = pd.concat([df_copy, pd.DataFrame([new_row])], ignore_index=True)

            if df_copy.loc[freq3_mask].shape[0] > 0:
                df_copy.loc[freq3_mask, 'Impressions'] += freq3_add
            else:
                # Если строки с частотой 3 нет, создаем новую
                new_row = df_copy[placement_mask].iloc[0].copy()
                new_row['Frequency'] = 3
                new_row['Impressions'] = freq3_add
                df_copy = pd.concat([df_copy, pd.DataFrame([new_row])], ignore_index=True)

# 4. Сохраняем файл
df_copy.to_csv('new_freq.csv', index=False)

Доступные Site Name:
1. Mail.ru
2. AdMile RU
3. Ozon RU
4. Mobidriven
5. Ekran.ru
6. MyTarget RU
7. Adspector RU
8. Innovation Lab RU
9. Digital Alliance RU
10. Trading Desk RU
11. Digital Max RU
12. MediaDesk RU
13. Mediatoday RU
14. GoTech RU


Введите номера Site Name, с которыми будем работать (через запятую):  10, 11


# Этот скрипт сравнивает выгрузки за прошлую неделю и текующую и проверяет, есть ли в выгрузке за текущую неделю значения меньше, чем за прошлую

In [39]:
import pandas as pd

# Читаем файлы с учетом правильного разделителя
df_prev = pd.read_csv('frequency_by_placements_1000248234_2025-04-01_2025-06-01_ver_7d02da9e24f7(1).csv', delimiter=';')
df_new = pd.read_csv('frequency_by_placements_1000248234_2025-05-01_2025-06-08_ver_e6d8d6f47b97.csv', delimiter=';')

# Убедимся, что нужные колонки есть
required_columns = ['Site Name', 'Placement Name', 'Frequency', 'Impressions']

for col in required_columns:
    if col not in df_prev.columns or col not in df_new.columns:
        raise KeyError(f"Колонка '{col}' не найдена в одном из файлов. Проверь названия колонок.")

# Обработка значений Frequency
for df in [df_prev, df_new]:
    df['Frequency'] = df['Frequency'].replace('100+', 100)
    df['Frequency'] = pd.to_numeric(df['Frequency'], errors='coerce')

# Оставляем только Placement Name, которые есть в обоих файлах
common_placements = set(df_prev['Placement Name']) & set(df_new['Placement Name'])

# Фильтруем файлы
df_prev = df_prev[df_prev['Placement Name'].isin(common_placements)]
df_new = df_new[df_new['Placement Name'].isin(common_placements)]

# Сравнение строк
issues = []

for index, row in df_prev.iterrows():
    site = row['Site Name']
    placement = row['Placement Name']
    frequency = row['Frequency']
    prev_impressions = row['Impressions']

    # Фильтр для новой недели
    match = df_new[
        (df_new['Site Name'] == site) &
        (df_new['Placement Name'] == placement) &
        (df_new['Frequency'] == frequency)
    ]

    if match.empty:
        # Если такой строки нет — пропускаем, так как этот Placement Name мы уже проверили, что он есть
        continue
    else:
        new_impressions = match['Impressions'].sum()
        if new_impressions < prev_impressions:
            issues.append(f"МЕНЬШЕ ПОКАЗОВ: {site} | {placement} | Frequency {frequency} | Предыдущие показы: {prev_impressions} | Новые показы: {new_impressions}")

# Вывод результата в консоль
if issues:
    print("\n⚠️ Найдены строки, где в новой неделе меньше показов, чем в прошлой:\n")
    for issue in issues:
        print(issue)
else:
    print("\n✅ Все ок! Во всех строках количество показов в новой неделе не меньше, чем в прошлой.")



⚠️ Найдены строки, где в новой неделе меньше показов, чем в прошлой:

МЕНЬШЕ ПОКАЗОВ: Trading Desk RU | 500340_trading_desk_slot_1_240x400 | Frequency 5 | Предыдущие показы: 41 | Новые показы: 40
МЕНЬШЕ ПОКАЗОВ: Trading Desk RU | 500340_trading_desk_slot_1_970x250 | Frequency 1 | Предыдущие показы: 3 | Новые показы: 2
МЕНЬШЕ ПОКАЗОВ: Trading Desk RU | 500340_trading_desk_slot_1_320x480_fullscreen | Frequency 2 | Предыдущие показы: 256 | Новые показы: 255
МЕНЬШЕ ПОКАЗОВ: Trading Desk RU | 500340_trading_desk_slot_1_480x320_fullscreen | Frequency 3 | Предыдущие показы: 1 | Новые показы: 0
МЕНЬШЕ ПОКАЗОВ: Trading Desk RU | 500340_trading_desk_slot_1_768x1024 | Frequency 1 | Предыдущие показы: 3 | Новые показы: 2
МЕНЬШЕ ПОКАЗОВ: Trading Desk RU | 500341_trading_desk_slot_2_240x400 | Frequency 2 | Предыдущие показы: 149 | Новые показы: 148
МЕНЬШЕ ПОКАЗОВ: Trading Desk RU | 500341_trading_desk_slot_2_1024x768 | Frequency 1 | Предыдущие показы: 2 | Новые показы: 1
